In [1]:
import spacy
import pandas as pd
from tqdm import tqdm
from spacy.tokens import DocBin

from sklearn.model_selection import train_test_split

# Загрузка датасета

In [2]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv",  encoding = 'utf8', sep = '\t')
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [3]:
df['sentiment'].value_counts()

sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

In [4]:
df = df[df['sentiment'] != 'neautral']
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


## Подготовка тренировочных и валидационных данных

In [5]:
X = df['review']
y = df['sentiment']

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = .8)

In [7]:
train_data = list(zip(X_train, y_train))
valid_data = list(zip(X_val, y_val))

In [8]:
train_data[0]

('Так и не пришли, продавец вернул деньги', 'negative')

In [9]:
valid_data[0]

('Короткие очень лямки, размер L на 44руб мал. На бирке - хлопок100%, по факту - синтетика. Электризуется так, что трещит, липнет к телу. Вряд ли можно это носить. ',
 'negative')

# Создание модели spacy

In [10]:
nlp = spacy.load('ru_core_news_md')

In [11]:
def make_docs(data):

    docs = []

    for doc, label in tqdm(nlp.pipe(data, as_tuples = True), total = len(data)):
        
        if label == 'negative':
            doc.cats['positive'] = 0
            doc.cats['negative'] = 1
        else:
            doc.cats['positive'] = 1
            doc.cats['negative'] = 0

        docs.append(doc)

    return docs

In [12]:
train_docs = make_docs(train_data)
valid_docs = make_docs(valid_data)

100%|██████████| 12000/12000 [01:02<00:00, 191.57it/s]


In [13]:
doc_bin = DocBin(docs = train_docs)
doc_bin.to_disk('train.spacy')

doc_bin = DocBin(docs = valid_docs)
doc_bin.to_disk('valid.spacy')

# Настройка конфига и обучение модели

In [14]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
! python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       52.76    0.53
  0     200         36.26       85.88    0.86
  0     400         26.34       88.53    0.89
  0     600         21.06       89.15    0.89
  0     800         20.05       90.22    0.90
  0    1000         16.55       90.67    0.91
  0    1200         15.68       91.22    0.91
  0    1400         15.24       91.72    0.92
  0    1600         13.81       92.03    0.92
  0    1800         13.61       92.46    0.92
  0    2000         13.12       92.79    0.93
  0    2200         12.45       92.93    0.93
  0    2400         12.40       93.03    0.93
  0    2600         11.85   

# Тестирование модели

In [16]:
pos_review = 'Приятная ткань, хороший пошив. Рекомендую, жалоб нет.'
neg_review = 'Это какой-то ужас!!1!! Оно разваливается прямо в руках!!!!! Верните деньги !111!!!!!'

In [17]:
nlp = spacy.load("output/model-best")

doc = nlp(pos_review)
print(doc.cats)

if doc.cats['positive'] >.5:
    print(f"the sentiment is positive")
else:
    print(f"the sentiment is negative")

doc = nlp(neg_review)
print(doc.cats)

if doc.cats['positive'] >.5:
    print(f"the sentiment is positive")
else:
    print(f"the sentiment is negative")

{'positive': 0.9895260334014893, 'negative': 0.010473988018929958}
the sentiment is positive
{'positive': 0.055960722267627716, 'negative': 0.9440392255783081}
the sentiment is negative
